## Setup MPI environment

In [1]:
import ipyparallel as ipp

nx = 12
ny = 12
nz = 79
nhalo = 3
backend = "numpy"

layout = (1, 1)
ntiles = 6
# spinup cluster of MPI-workers
num_ranks = ntiles * layout[0] * layout[1]

cluster = ipp.Cluster(engines="mpi", n=num_ranks).start_and_connect_sync()

# broadcast configuration to all workers
ar = cluster[:].push(
    {
        "ntiles": ntiles,
        "nx": nx,
        "ny": ny,
        "nz": nz,
        "nhalo": nhalo,
        "layout": layout,
        "backend": backend,
    }
)

# start executing cells on the workers in parallel from here on
%autopx

Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

%autopx enabled


[1:execute]
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[4], line 1
----> 1 metric_terms = MetricTerms(
      2     quantity_factory=quantity_factory,
      3     communicator=cs_communicator,
      4 )
      6 horizontal_data = HorizontalGridData.new_from_metric_terms(metric_terms)
      7 vertical_data = VerticalGridData.new_from_metric_terms(metric_terms)

File /usr/local/lib/python3.8/site-packages/ndsl/grid/generation.py:304, in MetricTerms.__init__(self, quantity_factory, communicator, grid_type, dx_const, dy_const, deglat, extdgrid, eta_file)
    297 self._area = None
    298 self._area_c = None
    299 (
    300     self._ks,
    301     self._ptop,
    302     self._ak,
    303     self._bk,
--> 304 ) = self._set_hybrid_pressure_coefficients(eta_file)
    305 self._ec1 = None
    306 self._ec2 = None

File /usr/local/lib/python3.8/site-packages/ndsl/grid/generatio

In [2]:
from mpi4py import MPI

mpi_comm = MPI.COMM_WORLD
mpi_rank = mpi_comm.Get_rank()
print(f"Hello from rank {mpi_rank}")

[stdout:0] Hello from rank 0


[stdout:1] Hello from rank 1


[stdout:2] Hello from rank 2


[stdout:3] Hello from rank 3


[stdout:5] Hello from rank 5


[stdout:4] Hello from rank 4


### Set up NDSL infrastructure

In [3]:
import gt4py.cartesian.gtscript as gtscript
from gt4py.cartesian.gtscript import PARALLEL, computation, interval

from ndsl.dsl.typing import Float, FloatField, FloatFieldIJ
from ndsl.comm.communicator import Communicator, CubedSphereCommunicator, TileCommunicator
from ndsl.dsl.stencil import StencilFactory, GridIndexing
from ndsl.initialization import SubtileGridSizer
from ndsl.initialization.allocator import QuantityFactory
from ndsl.quantity import Quantity
from ndsl.comm.partitioner import CubedSpherePartitioner, TilePartitioner
from ndsl.constants import X_DIM, Y_DIM, Z_DIM
from ndsl.dsl.stencil_config import CompilationConfig, StencilConfig
from ndsl.dsl.dace.wrapped_halo_exchange import WrappedHaloUpdater
from ndsl.grid import (
    DampingCoefficients,
    DriverGridData,
    GridData,
    MetricTerms,
    direct_transform,
)
from ndsl.grid.helper import (
    AngleGridData,
    ContravariantGridData,
    HorizontalGridData,
    VerticalGridData,
)
from ndsl.namelist import Namelist, NamelistDefaults

import pyFV3
import pyFV3.initialization.analytic_init as analytic_init
from dataclasses import fields
import yaml
from datetime import timedelta

[output:1]

[output:0]

[output:4]

[output:2]

[output:5]

...

[stdout:3] 2024-04-16 17:36:23|INFO|rank 3|ndsl.logging:Constant selected: ConstantVersions.GFS


[stdout:1] 2024-04-16 17:36:23|INFO|rank 1|ndsl.logging:Constant selected: ConstantVersions.GFS


[stdout:0] 2024-04-16 17:36:23|INFO|rank 0|ndsl.logging:Constant selected: ConstantVersions.GFS


[stdout:2] 2024-04-16 17:36:23|INFO|rank 2|ndsl.logging:Constant selected: ConstantVersions.GFS


%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

In [4]:
partitioner = CubedSpherePartitioner(TilePartitioner(layout))
cs_communicator = CubedSphereCommunicator(mpi_comm, partitioner)


sizer = SubtileGridSizer.from_tile_params(
    nx_tile=nx,
    ny_tile=ny,
    nz=nz,
    n_halo=nhalo,
    extra_dim_lengths={},
    layout=layout,
    tile_partitioner=partitioner.tile,
    tile_rank=cs_communicator.tile.rank,
)

# useful for easily allocating distributed data storages (fields)
quantity_factory = QuantityFactory.from_backend(sizer=sizer, backend=backend)

compilation_config = CompilationConfig(backend=backend, communicator=cs_communicator)

stencil_config = StencilConfig(compare_to_numpy=False, compilation_config=compilation_config)

grid_indexing = GridIndexing.from_sizer_and_communicator(sizer=sizer, comm=cs_communicator)

stencil_factory = StencilFactory(config=stencil_config, grid_indexing=grid_indexing)

### Grid

In [5]:
metric_terms = MetricTerms(
    quantity_factory=quantity_factory,
    communicator=cs_communicator,
    eta_file="eta79.nc",
)

horizontal_data = HorizontalGridData.new_from_metric_terms(metric_terms)
vertical_data = VerticalGridData.new_from_metric_terms(metric_terms)
contravariant_data = ContravariantGridData.new_from_metric_terms(metric_terms)
angle_data = AngleGridData.new_from_metric_terms(metric_terms)
grid_data = GridData(
    horizontal_data=horizontal_data,
    vertical_data=vertical_data,
    contravariant_data=contravariant_data,
    angle_data=angle_data,
)
damping_coefficients = DampingCoefficients.new_from_metric_terms(metric_terms)

driver_grid_data = DriverGridData.new_from_metric_terms(metric_terms)

AlreadyDisplayedError: 6 errors

## Init dycore

In [6]:
from datetime import datetime, timedelta
from math import floor
import yaml

yaml_config = "c12_baroclinic.yaml"
runtime = {}
runtime["days"] = 0.
runtime["hours"] = 0.
runtime["minutes"] = 0.
runtime["seconds"] = 0.

with open(yaml_config) as f:
    raw_config = yaml.safe_load(f)

for key in runtime.keys():
    if key in raw_config.keys():
        runtime[key] = raw_config[key]

total_time = timedelta(
    days=runtime["days"], hours=runtime["hours"], minutes=runtime["minutes"], seconds=runtime["seconds"]
)

config = pyFV3.DynamicalCoreConfig.from_yaml(yaml_config)
config.ntiles = 6

timestep = timedelta(seconds=config.dt_atmos)

n_steps = floor(total_time.total_seconds() / timestep.total_seconds())

state = analytic_init.init_analytic_state(
    analytic_init_case="baroclinic",
    grid_data=grid_data,
    quantity_factory=quantity_factory,
    adiabatic=False,
    hydrostatic=False,
    moist_phys=True,
    comm=cs_communicator,
)

AlreadyDisplayedError: 6 errors

In [7]:
dycore = pyFV3.DynamicalCore(
    comm=cs_communicator,
    grid_data=grid_data,
    quantity_factory=quantity_factory,
    stencil_factory=stencil_factory,
    damping_coefficients=damping_coefficients,
    config=config,
    state=state,
    phis=state.phis,
    timestep=timestep,
)

AlreadyDisplayedError: 6 errors

## And here we can start looping through

In [8]:
for step in range(n_steps):
    dycore.step_dynamics(state=state)

AlreadyDisplayedError: 6 errors